In [ ]:
#Transient simulation of osmotic flows through a membrane 
#From P. Bacchin (published in An energy map model for colloid transport. Chemical Engineering Science. 2017, http://dx.doi.org/10.1016/j.ces.2016.10.024)
#Adapted from http://www.ctcms.nist.gov/fipy/examples/diffusion/generated/examples.diffusion.coupled.html
#You are free to use this code but :
#If you use this code for your research, please cite : http://dx.doi.org/10.1016/j.ces.2016.10.024
#If you are improving significantly this code, please send an updated copy to patrice.bacchin@univ-tlse3.fr
#Enjoy osmotic flows !
from pylab import *
from fipy import *
close("all")

#data
#permeability of the membrane_m and the polarisation layer_pl
perm_m=1.
perm_pl=100.
#partition and maximum interfacial pressure
part =0.1
piw_m =-log(part)
#composite layers (polarisation-pl1 exclusion_ex2 membrane_m exclusion_ex2 polarisation_pl2)
d_pl1=1./3.2
d_ex1=0.1/3.2
d_m=1./3.2
d_ex2=0.1/3.2
d_pl2=1./3.2
#surface ratio of the membrane over vertical tube (geometry of the Abbe Nollet exp.)
S=1000000.
#Adimensionnal fluid density * gravity 
rog=1.e-5

#time steps
dt=1.e-3
steps=201

#meshing
nx=500.
Lx=1.
m = Grid1D(nx=nx, Lx=Lx)

#variables : initialisation & boundary conditions
phi_init=0.
phi_c=0.1 
phi_d=0.
pre_init=10.
pre_c=10.
pre_d=10.
phi = CellVariable(mesh=m, hasOld=True, value=phi_init)
phi.constrain(phi_c, m.facesLeft)
phi.constrain(phi_d, m.facesRight)
pre = CellVariable(mesh=m, hasOld=True, value=pre_init)
pre.constrain(pre_c, m.facesLeft)
pre.constrain(pre_d, m.facesRight)

#interfacial pressure gradient (migration velocity)
X = m.faceCenters[0]
um = FaceVariable(mesh=m, value=0)
um.setValue(((piw_m*pi*0.5*cos(pi*((X-d_pl1)/d_ex1-0.5))/d_ex1),),where=(d_pl1 <= X) & (X <= d_pl1+d_ex1))
um.setValue(((-piw_m*pi*0.5*cos(pi*((X-(d_pl1+d_ex1+d_m))/(d_ex2)-0.5))/(d_ex2)),),where=((d_pl1+d_ex1+d_m) <= X) & (X <= (d_pl1+d_ex1+d_m+d_ex2)))

#local permeability (interfacial pressure gradient)
perm = FaceVariable(mesh=m, value=perm_pl)
perm.setValue((perm_m,),where=((X >= (d_pl1+d_ex1)) & (X <= (d_pl1+d_ex1+d_m))))

#mass balance equation
eq0 = TransientTerm() + ConvectionTerm((-perm * (pre.faceGrad + (phi.arithmeticFaceValue*um)))-um) == DiffusionTerm(coeff=1)
#fluid continuity (div(Pe)=0
eq1 = (DiffusionTerm(coeff = perm, var=pre) + ((perm * (phi.arithmeticFaceValue*um)).divergence) == 0) 

#transient solving
step=0
for step in range(steps): 
phi.updateOld()
pre.updateOld()
res0 = res1 = 1e100
while max(res0, res1) > 0.001:
res1= eq1.sweep(var=pre, dt=dt)
res0 = eq0.sweep(var=phi, dt=dt)
Pe=(-perm * (pre.faceGrad + (phi.arithmeticFaceValue*um))) 
pre_c=pre_c-(mean(Pe.value)*S*rog)*dt
pre_d=pre_d+(mean(Pe.value)*S*rog)*dt
pre.constrain(pre_c, m.facesLeft)
pre.constrain(pre_d, m.facesRight)
